In [ ]:
%%bash

# Test Run with 10 designs 
target="PDL1.json"
target_name="${target%.json}"  # Strip the .json extension
target_path="/workspace/settings_target/$target"
log_dir="/workspace/outputs/$target_name"
log_file="$log_dir/${target_name}-bindcraft_log.txt"

# Ensure log directory exists
mkdir -p "$log_dir"

echo "[INFO] Checking target settings file..."
if [ ! -f "$target_path" ]; then
  echo "[ERROR] Target file '$target' not found in /workspace/settings_target/"
  exit 1
fi

echo "[INFO] Starting BindCraft with target '$target'"
echo "[INFO] Logging output to $log_file"

nohup python /app/bindcraft/bindcraft.py \
  --settings "$target_path" \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > "$log_file" 2>&1 &

echo "[INFO] BindCraft is now running in the background (PID=$!)"

In [ ]:
!tail -n 50 -f /workspace/outputs/${log_file}

In [1]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload(accept='.json', multiple=False)
display(uploader)

FileUpload(value=(), accept='.json', description='Upload')

In [44]:
import json
import os
from glob import glob
from pathlib import Path
from functools import partial
import ipywidgets as widgets
from IPython.display import display

def parse_target_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def save_target_json(target_folder, filename, json_target_path, data):
    base_dir = os.path.dirname(json_target_path)
    target_filepath = os.path.join(base_dir, target_folder)
    os.makedirs(target_filepath, exist_ok=True)
    
    full_path = os.path.join(target_filepath, filename)
    print(f"[DEBUG] Writing file to: {full_path}")

    with open(full_path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f'Data saved to {full_path}')

def update_data(data, widget_dict):
    updated_data = {}
    for key, widget in widget_dict.items():
        w = widget.value
        #Convert back to a list if the original data was a list
        if isinstance(data[key], list):
            try:
                w = json.loads(w)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for key '{key}'. Keeping original value.")
                w = data[key]
        updated_data[key] = w
    return updated_data     

def on_save_clicked(button, data, widget_dict, target_folder_widget, filename_widget, json_target_path):
    print("[DEBUG] Save button clicked")
    target_folder = target_folder_widget.value.strip()
    filename = filename_widget.value.strip()
    updated_data = update_data(data, widget_dict)
    save_target_json(target_folder, filename, json_target_path, updated_data)
    print(f'Saved parameters to {filename} in folder {target_folder}')


#UI Builder

def target_editor_widget(data):
    widget_dict = {}
    for k, v in data.items():
         
        if isinstance(v, list):
            widget_dict[k] = widgets.Text(value=str(v), description=f'{k}:',
                                          style={'description_width': 'initial'})
    
        elif isinstance(v, int):
            widget_dict[k] = widgets.IntText(value=v, description=f'{k}:',
                                             style={'description_width': 'initial'})

        else:
            widget_dict[k] = widgets.Text(value=v, description=f'{k}:',
                                          style={'description_width': 'initial'})
        
    return widget_dict

def settings_widget(dirs = ['/workspace/settings_filters', '/workspace/settings_advanced']):
    for d in dirs:
        file_list = sorted(glob(f'{d}/*.json'))
        filenames = [os.path.basename(file) for file in file_list]
        
        #Create the dropdowns
        file_dropdown = widgets.Dropdown(
        options=filenames,
        description=f'{os.path.basename(d)} files:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
        )
        display(file_dropdown)

def main_launch_editor(json_target_path='/workspace/settings_target/PDL1.json'):
    data = parse_target_json(json_target_path)

    target_folder_widget = widgets.Text(value=Path(json_target_path).stem,
                                    description='Subfolder to save json file to',
                                    style={'description_width': 'initial'}
                                   )
    filename_widget = widgets.Text(value=os.path.basename(json_target_path),
                               description='File Name:',
                               layout=widgets.Layout(width='50%')
                               )
    
    #Editable Field widgets
    widget_dict = target_editor_widget(data)

    save_button = widgets.Button(description="Save Changes", button_style='success')
    save_button.on_click(partial(on_save_clicked,
                             data=data,
                             widget_dict=widget_dict,
                             target_folder_widget=target_folder_widget,
                             filename_widget=filename_widget,
                             json_target_path=json_target_path
    ))
    #Display widgets
    display(filename_widget)
    display(target_folder_widget)
    for widget in widget_dict.values():
        display(widget)
    display(save_button)



In [45]:
main_launch_editor('/home/yarrow/projects/bindcraft-runpod/settings_target/PDL1.json')

Text(value='PDL1.json', description='File Name:', layout=Layout(width='50%'))

Text(value='PDL1', description='Subfolder to save json file to', style=TextStyle(description_width='initial'))

Text(value='/workspace/outputs/PDL1', description='design_path:', style=TextStyle(description_width='initial')…

Text(value='PDL1-Binder', description='binder_name:', style=TextStyle(description_width='initial'))

Text(value='/workspace/inputs/PDL1.pdb', description='starting_pdb:', style=TextStyle(description_width='initi…

Text(value='A', description='chains:', style=TextStyle(description_width='initial'))

Text(value='56', description='target_hotspot_residues:', style=TextStyle(description_width='initial'))

Text(value='[65, 150]', description='lengths:', style=TextStyle(description_width='initial'))

IntText(value=10, description='number_of_final_designs:', style=DescriptionStyle(description_width='initial'))

Button(button_style='success', description='Save Changes', style=ButtonStyle())

In [ ]:
from glob import glob
import os
import ipywidgets as widgets
from IPython.display import display
from functools import partial
import subprocess
from string import Template

# GLOBAL SETTINGS
selected_paths_holder = {}
SETTINGS_FILE_PATH = '/home/yarrow/projects/bindcraft-runpod/'
BINDCRAFT_RUN_TEMPLATE_PATH = "/home/yarrow/projects/bindcraft-runpod/bindcraft_run_template.sh"
BINDCRAFT_SCRIPT_OUTPUT_PATH = "/home/yarrow/projects/bindcraft-runpod/bindcraft_run.sh"

#SETTINGS_DIRS = [
#    '/workspace/settings_filters',
#    '/workspace/settings_advanced',
#]
# For Runpod: '/workspace/'

def settings_widget(dirs: list) -> dict:
    settings_widget_dict = {}
    for d in dirs:
        file_list = sorted(glob(f'{d}/*.json'))
        filenames = [os.path.basename(file) for file in file_list]
        
        #Create the dropdowns
        file_dropdown = widgets.Dropdown(
        options=filenames,
        description=f'{os.path.basename(d)} files:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
        )
        #Store the widgets in a dictionary
        settings_widget_dict[d] = file_dropdown
    #print(f"[DEBUG] Settings widget dictionary: {settings_widget_dict}")
    return settings_widget_dict
        
def on_submit_clicked(button, settings_widget_dict: dict) -> None:
    selected_paths = {os.path.basename(k): dd.value for k, dd in settings_widget_dict.items()}
    selected_paths_holder.clear()
    selected_paths_holder.update(selected_paths)
    
    for k, v in selected_paths_holder.items():
        print(f'{k}: {v}')
   
def run_bindcraft(SETTINGS_FILE_PATH: str, selected_paths_holder: dict, target_path: str) -> None:
 
    for folder, file in selected_paths_holder.items():
        full_path = os.path.join(SETTINGS_FILE_PATH, folder, file)
        selected_paths_holder[folder] = full_path
    
    try:
        filters_path = selected_paths_holder['settings_filters']
        advanced_path = selected_paths_holder['settings_advanced']
        selected_paths_holder['settings_target'] = target_path
    except KeyError:
        print(f"Please click on the Submit button first to select settings files.")
        return
    
    if not filters_path or not advanced_path or not target_path:
        print(f'Please select both filters and advanced settings files.')
        return
    
    print(filters_path, advanced_path, target_path)
    with open(BINDCRAFT_RUN_TEMPLATE_PATH, 'r') as f:
        template = Template(f.read())
    
    # Replace placeholders in the template
    template = template.substitute(
    
    FILTERS_FILE_PATH=filters_path,
    ADVANCED_FILE_PATH=advanced_path,
    TARGET_FILE_PATH=target_path,
    TARGET_FILE_NAME=os.path.basename(target_path),
    TARGET_NAME=os.path.splitext(os.path.basename(target_path))[0],
    LOG_DIR=f"/workspace/outputs/{os.path.splitext(os.path.basename(target_path))[0]}"
)
    # Write the modified template to the output script
    with open(BINDCRAFT_SCRIPT_OUTPUT_PATH, 'w') as output_script:
        output_script.write(template)

    os.chmod(BINDCRAFT_SCRIPT_OUTPUT_PATH, 0o755)
    print(f"BindCraft script written to {BINDCRAFT_SCRIPT_OUTPUT_PATH}")

    try:
        subprocess.run(["bash", BINDCRAFT_SCRIPT_OUTPUT_PATH], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with exit code {e.returncode}")


def main(dirs) -> list:
    settings_widget_dict = settings_widget(dirs)
    submit_button = widgets.Button(description = 'Submit Selected settings',
                                   button_style = 'success',
                                   layout = widgets.Layout(width='30%')
                                   )
    submit_button.on_click(partial(on_submit_clicked, 
                                   settings_widget_dict=settings_widget_dict,
                                   )
                            )
    #Display Widgets 
    for directory, widget in settings_widget_dict.items:
        print(f"Displaying dropdown for {directory}")
        display(widget)
    display(submit_button)


        


In [87]:

dirs =['/home/yarrow/projects/bindcraft-runpod/settings_filters',
       '/home/yarrow/projects/bindcraft-runpod/settings_advanced'] 
#['/workspace/settings_filters', '/workspace/settings_advanced']
settings_widget_dict = settings_widget(dirs)

submit_button = widgets.Button(description = 'Submit Selected settings',
                                   button_style = 'success',
                                   layout = widgets.Layout(width='30%')
                                   )  
selected_paths = submit_button.on_click(partial(on_submit_clicked, 
                                   settings_widget_dict=settings_widget_dict,
                                   )
                )
#Display Widgets 
for directory, widget in settings_widget_dict.items():
       print(f"Displaying dropdown for {directory}")
       display(widget)

display(submit_button)



Displaying dropdown for /home/yarrow/projects/bindcraft-runpod/settings_filters


Dropdown(description='settings_filters files:', layout=Layout(width='50%'), options=('default_filters.json', '…

Displaying dropdown for /home/yarrow/projects/bindcraft-runpod/settings_advanced


Dropdown(description='settings_advanced files:', layout=Layout(width='50%'), options=('betasheet_4stage_multim…

Button(button_style='success', description='Submit Selected settings', layout=Layout(width='30%'), style=Butto…

In [89]:
#print(selected_paths_holder)
run_bindcraft(SETTINGS_FILE_PATH, selected_paths_holder, '/home/yarrow/projects/bindcraft-runpod/settings_target/PDL1.json')

/home/yarrow/projects/bindcraft-runpod/settings_filters/default_filters.json /home/yarrow/projects/bindcraft-runpod/settings_advanced/betasheet_4stage_multimer.json /home/yarrow/projects/bindcraft-runpod/settings_target/PDL1.json
BindCraft script written to /home/yarrow/projects/bindcraft-runpod/bindcraft_run.sh


In [94]:
import sys
dir = '/home/yarrow/projects/bindcraft-runpod/functions/'
sys.path.append(dir)

from UI_launch import launch_all_ui

launch_all_ui()


ImportError: cannot import name 'main_launch_editor' from 'ui_target_editor' (/home/yarrow/projects/bindcraft-runpod/functions/ui_target_editor.py)